
opengl context?


In [ ]:

#include <iostream>
//#include <glad/glad.h>
#include <SDL.h>


#if __cplusplus
extern "C" {
#endif


int *createWindow(void) {
	SDL_Window* window = nullptr;
	
	int screen_width = 1000;
	int screen_height = 800;

	if (SDL_Init(SDL_INIT_VIDEO) < 0) {
		std::cout << "SDL failed initialization. " << SDL_GetError();
		return NULL;
	} else {
		std::cout << "SDL successfully initialized.";
	}

	SDL_GL_SetAttribute(SDL_GL_CONTEXT_MAJOR_VERSION, 4);
	SDL_GL_SetAttribute(SDL_GL_CONTEXT_MINOR_VERSION, 1);
	SDL_GL_SetAttribute(SDL_GL_CONTEXT_PROFILE_MASK, SDL_GL_CONTEXT_PROFILE_CORE);
	SDL_GL_SetAttribute(SDL_GL_DOUBLEBUFFER, 1);
	SDL_GL_SetAttribute(SDL_GL_DEPTH_SIZE, 24);

	window = SDL_CreateWindow("C++ SDL2 Window | OpenGL", 
			0, 
			0, 
			screen_width, 
			screen_height, 
			SDL_WINDOW_SHOWN | SDL_WINDOW_OPENGL);
	SDL_GLContext context;
	context = SDL_GL_CreateContext(window);

	//gladLoadGLLoader(SDL_GL_GetProcAddress);

	bool done = false;

  return (int *)window;
}

#if __cplusplus
}
#endif



test opengl renderer?


In [ ]:


let beginFrame, endFrame
async function doFrame(window) {
  if(!beginFrame) {
    frameLoader = await importer.import('opengl frame')
    beginFrame = frameLoader.beginFrame
    endFrame = frameLoader.endFrame
  }
  beginFrame(window)

  endFrame(window)
}

async function testOpenGL() {
  const {createWindow} = await importer.import('opengl context')
  let window = createWindow()
  console.log(window)
  setInterval(doFrame.bind(null, window), 1000)
}

module.exports = testOpenGL



opengl frame?


In [ ]:
#include <vector>
#include <cmath>
#include <SDL.h>

class Point {
  public:
    double x, y, z;

    Point() {
        this->x = 0;
        this->y = 0;
        this->z = 0;
    }

    Point(double x, double y, double z) {
        this->x = x;
        this->y = y;
        this->z = z;
    }

    Point(double x, double y) {
        this->x = x;
        this->y = y;
        this->z = 0;
    }

    double operator[](int i) const {
        if (i == 0) return this->x;
        if (i == 1) return this->y;
        return this->z;
    }

    double& operator[](int i) {
        if (i == 0) return this->x;
        if (i == 1) return this->y;
        return this->z;
    }
};

typedef std::vector<double> Vector;
typedef std::vector<Vector> Matrix;

Matrix dot(const Matrix& a, const Matrix& b) {
    Matrix result = Matrix(a.size(), Vector(b[0].size(), 0));
    for (int i=0; i<a.size(); i++) {
        for (int j=0; j<b[0].size(); j++) {
            for (int k=0; k<b.size(); k++) {
                result[i][j] += a[i][k] * b[k][j];
            }
        }
    }
    return result;
}

Point transform(const Matrix& matrix, const Point& point) {
    Matrix p = {{point.x}, {point.y}, {point.z}};
    Matrix r = dot(matrix, p);
    return Point(r[0][0], r[1][0], r[2][0]);
}

Point translate(const Point& shift, const Point& point) {
    return Point(
        point.x + shift.x,
        point.y + shift.y,
        point.z + shift.z
    );
}

void connect(SDL_Renderer* const renderer, const std::vector<Point> &points, int i, int j) {
    SDL_RenderDrawLine(
        renderer,
        points[i].x,
        points[i].y,
        points[j].x,
        points[j].y
    );
}

Matrix getRotationMatrix() {
    double alpha = 0.001;
    Matrix rotationX = {
        {1, 0, 0},
        {0, cos(alpha), -sin(alpha)},
        {0, sin(alpha), cos(alpha)}
    };

    double beta = 0.002;
    Matrix rotationY = {
        {cos(beta), 0, sin(beta)},
        {0, 1, 0},
        {-sin(beta), 0, cos(beta)}
    };

    double gamma = 0.003;
    Matrix rotationZ = {
        {cos(gamma), -sin(gamma), 0},
        {sin(gamma), cos(gamma), 0},
        {0, 0, 1}
    };

    return dot(rotationZ, dot(rotationY, rotationX));
}
  

#if __cplusplus
extern "C" {
#endif


int WIDTH = 1000;
int HEIGHT = 800;

static bool first = false;
static SDL_Renderer* renderer;
std::vector<Point> points;
Matrix rotationXYZ;

Point screenShift;
Point screenShiftOpposite;

void beginFrame(int *window) {

  if(!first) {
    first = true;
    
    renderer = SDL_CreateRenderer(
        (SDL_Window *)window,
        -1,
        SDL_RENDERER_ACCELERATED
    );

    points = {
        Point(-1, 1, 1),
        Point(1, 1, 1),
        Point(1, -1, 1),
        Point(-1, -1, 1),
        Point(-1, 1, -1),
        Point(1, 1, -1),
        Point(1, -1, -1),
        Point(-1, -1, -1)
    };

    screenShift = Point(WIDTH / 2, HEIGHT / 2);
    screenShiftOpposite = Point(-WIDTH / 2, -HEIGHT / 2);
    int scale = 200;

    for (Point& p : points) {
        p.x = (scale * p.x + screenShift.x);
        p.y = (scale * p.y + screenShift.y);
        p.z = (scale * p.z + screenShift.z);
    }

    rotationXYZ = getRotationMatrix();
  }


  for (Point &p : points) {
    p = translate(screenShiftOpposite, p);
    p = transform(rotationXYZ, p);
    p = translate(screenShift, p);
  }

  SDL_SetRenderDrawColor(renderer, 255, 255, 255, 255);
  SDL_RenderClear(renderer);
  SDL_SetRenderDrawColor(renderer, 0, 0, 0, 255);
  for (int i=0; i<4; i++) {
      connect(renderer, points, i, (i + 1) % 4);
      connect(renderer, points, i + 4, ((i + 1) % 4) + 4);
      connect(renderer, points, i, i + 4);
  }
  SDL_RenderPresent(renderer);
  SDL_Delay(3);
}

void endFrame(int *window) {
  SDL_GL_SwapWindow((SDL_Window *)window);
}


#if __cplusplus
}
#endif

